In [ ]:
#!/usr/bin/env python3
"""
2D particle simulation in a 1 cm (x) by 6 cm (y) rectangular domain with periodic boundaries.
Grid: 1 cell across (x) and 6 cells tall (y), cell size = 1 cm.
Each cell center emits 4 particles at t=0 with velocities: up, down, right, left.
Colors: up/down/right = blue, left = red.

Output: tries MP4 (ffmpeg) first, falls back to GIF (pillow) if MP4 is unavailable.
Dependencies: numpy, matplotlib
"""

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import sys
sys.path.insert(0, "..")
import nsm_plots.plot_functions as pf  # applies font/tick/axis settings from plot_functions

# ----------------------------
# Parameters (edit these)
# ----------------------------
v = 0.5                 # cm/s (particle speed)
dt = 0.05               # s (simulation time step per frame)
fps = 30                # frames per second for the saved animation
duration_seconds = 12.0 # total simulated duration (s)

# ----------------------------
# Domain and grid
# ----------------------------
Lx = 6.0  # cm
Ly = 1.0  # cm
cell_size = 1.0  # cm

nx = int(Lx / cell_size)  # should be 1
ny = int(Ly / cell_size)  # should be 6

# Cell centers (emission points)
# With nx = 1, x center is always 0.5
x_centers = (np.arange(nx) + 0.5) * cell_size
y_centers = (np.arange(ny) + 0.5) * cell_size

# ----------------------------
# Initialize particles
# ----------------------------
# For each cell center, spawn 4 particles: up, down, right, left
positions = []
velocities = []
colors = []

for xc in x_centers:
    for yc in y_centers:
        # Up (+y)
        positions.append([xc, yc])
        velocities.append([0.0, +v])
        colors.append("blue")

        # Down (-y)
        positions.append([xc, yc])
        velocities.append([0.0, -v])
        colors.append("blue")

        # Right (+x)
        positions.append([xc, yc])
        velocities.append([+v, 0.0])
        colors.append("blue")

        # Left (-x)
        positions.append([xc, yc])
        velocities.append([-v, 0.0])
        colors.append("red")

pos = np.array(positions, dtype=float)   # shape (N, 2)
vel = np.array(velocities, dtype=float)  # shape (N, 2)
colors = np.array(colors)

N = pos.shape[0]
n_frames = int(np.round(duration_seconds / dt))

# ----------------------------
# Periodic boundary condition helper
# ----------------------------
def apply_periodic(p):
    # Map positions into [0, L) using modulo
    p[:, 0] = p[:, 0] % Lx
    p[:, 1] = p[:, 1] % Ly
    return p

# ----------------------------
# Matplotlib figure setup
# ----------------------------
# Font settings
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'serif'
mpl.rc('text', usetex=True)

# Tick settings
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.major.pad'] = 8
mpl.rcParams['xtick.minor.size'] = 4

mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.size'] = 7
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.size'] = 4
mpl.rcParams['ytick.minor.width'] = 2

# Axis linewidth
mpl.rcParams['axes.linewidth'] = 2

# Tick direction and enabling ticks on all sides
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

# Set DPI for high-resolution figure. Ensure > 500 px on shorter side.
# With figsize=(10, 3), height in pixels is 3*DPI; so set DPI >= 167 for 500px.
fig_dpi = 500  # This gives 600x2000px (height x width) for figsize (10,3)
fig, ax = plt.subplots(figsize=(10, 3), dpi=fig_dpi)
ax.set_aspect("equal", adjustable="box")
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
ax.set_xlabel(r"$z\,[\mathrm{cm}]$")
ax.set_ylabel(r"$x\,[\mathrm{cm}]$")
# ax.set_title("Periodic particle motion in a 1 cm × 6 cm domain")

# Domain outline (rectangle) -- draw only three sides (hide right vertical side)
# We'll draw: bottom, left, and top as individual lines

# Bottom: (0,0) to (Lx,0)
ax.plot([0, Lx], [0, 0], color='k', linewidth=2, zorder=2)
# Left: (0,0) to (0,Ly)
ax.plot([0, 0], [0, Ly], color='k', linewidth=2, zorder=2)
# Top: (0,Ly) to (Lx,Ly)
ax.plot([0, Lx], [Ly, Ly], color='k', linewidth=2, zorder=2)

# Make sure right spine is not visible
ax.spines['right'].set_visible(False)

# Optional: also remove ticks from right spine for clarity
ax.yaxis.set_ticks_position('left')

# Scatter plot for particles
scat = ax.scatter(pos[:, 0], pos[:, 1], s=35, c=colors)
pf.apply_custom_settings(ax)

plt.show()

# ----------------------------
# Animation functions
# ----------------------------
def init():
    scat.set_offsets(pos)
    # Reset axis limits and labels
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, Ly)
    ax.set_xlabel(r"$z$")
    ax.set_ylabel(r"$x$")
    return scat,

def update(frame_idx):
    global pos, scat
    # Remove the old scatter plot
    scat.remove()
    # Advance positions
    pos = pos + vel * dt
    pos = apply_periodic(pos)
    # Create a new scatter plot for updated positions
    scat_new = ax.scatter(pos[:, 0], pos[:, 1], s=35, c=colors)
    # Reset axis and labels for each frame in case they've changed
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, Ly)
    ax.set_xlabel(r"$z$")
    ax.set_ylabel(r"$x$")
    # Needed so the *reference* updates in subsequent frames
    globals()['scat'] = scat_new
    return scat_new,

anim = animation.FuncAnimation(
    fig,
    update,
    init_func=init,
    frames=n_frames,
    interval=1000.0 / fps,
    blit=True
)

# ----------------------------
# Save animation: MP4 preferred, GIF fallback
# ----------------------------
mp4_name = "particles_1x6cm_periodic.mp4"
gif_name = "particles_1x6cm_periodic.gif"

saved = False

# Try MP4 with ffmpeg if available
try:
    if animation.writers.is_available("ffmpeg"):
        Writer = animation.writers["ffmpeg"]
        writer = Writer(fps=fps, metadata={"artist": "matplotlib"}, bitrate=1800)
        anim.save(mp4_name, writer=writer, dpi=fig_dpi)  # pass dpi for quality
        print(f"Saved MP4: {mp4_name}")
        saved = True
    else:
        print("ffmpeg writer not available in matplotlib. Will try GIF instead.")
except Exception as e:
    print(f"MP4 save failed: {e}")
    print("Will try GIF instead.")

# Fallback to GIF using PillowWriter
if not saved:
    try:
        writer = animation.PillowWriter(fps=fps)
        anim.save(gif_name, writer=writer, dpi=fig_dpi)  # pass dpi for quality
        print(f"Saved GIF: {gif_name}")
        saved = True
    except Exception as e:
        print(f"GIF save failed: {e}")
        saved = False

plt.close(fig)

if not saved:
    raise RuntimeError("Failed to save animation as MP4 or GIF. Check your matplotlib writers and dependencies.")